In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip --no-check-certificate

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
#df_ratings = df_ratings.groupby('user').filter(lambda grp: len(grp) >= 200)
#df_ratings.groupby('isbn').filter(lambda grp: len(grp) >= 100)

#counts1 = df_ratings['user'].value_counts()
#df_ratings = df_ratings[df_ratings['user'].isin(counts1[counts1 >= 200].index)]
#counts = df_ratings['rating'].value_counts()
#df_ratings = df_ratings[df_ratings['rating'].isin(counts[counts >= 100].index)]

df = df_ratings

counts1 = df['user'].value_counts()
counts2 = df['isbn'].value_counts()

df = df[~df['user'].isin(counts1[counts1 < 200].index)]
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]

df = pd.merge(right=df, left = df_books, on="isbn")

df = df.drop_duplicates(["title", "user"])

piv = df.pivot(index='title', columns='user', values='rating').fillna(0) 


In [ ]:
matrix = piv.values

knn_model = NearestNeighbors(metric='cosine', algorithm='brute', p=2)
knn_model.fit(matrix)

In [ ]:
titles = list(piv.index.values)
print(len(titles))

In [8]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  distances, indices = knn_model.kneighbors(piv.loc[book].values.reshape(1, -1), len(titles), True)
  recommended_books = [book, sum([[[piv.index[indices.flatten()[i]], distances.flatten()[i]]] for i in range(5, 0, -1)], [])]

  return recommended_books

In [ ]:
print(get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))"))

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    print(i)
    recommends[1][1][0]
    if recommends[1][i][0] not in recommended_books:
      print("title failure")
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      print("distance failure")
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()